In [228]:
import pandas as pd
import matplotlib as plt
import numpy as np
from pathlib import Path

In [229]:
edmPath = Path("edm.csv")
vanPath = Path("van.csv")
nhlPath = Path("nhl.csv")

edm = pd.read_csv(edmPath)
van = pd.read_csv(vanPath)
nhl = pd.read_csv(nhlPath)

edmClean = edm[["Game", "GF", "GA", "SH%", "SV%", "PDO", "SF", "SA", "SF%", "xGF", "xGA", "xGF%", "GF%"]]
vanClean = van[["Game", "GF", "GA", "SH%", "SV%", "PDO", "SF", "SA", "SF%", "xGF", "xGA", "xGF%", "GF%"]]

edmClean.insert(1, "Result", np.where(edmClean["GF"] > edmClean["GA"], "W", "L"))
vanClean.insert(1, "Result", np.where(vanClean["GF"] > vanClean["GA"], "W", "L"))

edmClean.head()

,Game,Result,GF,GA,SH%,SV%,PDO,SF,SA,SF%,xGF,xGA,xGF%,GF%
0,"2023-10-11 - Oilers 1, Canucks 8",L,1,8,3.70,75.00,0.787,27,32,45.76,2.97,3.83,43.73,11.11
1,"2023-10-14 - Canucks 4, Oilers 3",L,3,4,7.50,75.00,0.825,40,16,71.43,5.16,2.07,71.31,42.86
2,"2023-10-17 - Oilers 6, Predators 1",W,6,1,20.00,97.73,1.177,30,44,40.54,3.06,3.07,49.91,85.71
3,"2023-10-19 - Oilers 1, Flyers 4",L,1,4,4.35,87.50,0.918,23,32,41.82,2.07,2.79,42.59,20.00
4,"2023-10-21 - Jets 3, Oilers 2",L,2,3,5.00,88.89,0.939,40,27,59.70,3.67,2.33,61.20,40.00


In [230]:
edmTotals = edmClean.mean(numeric_only=True)
vanTotals = vanClean.mean(numeric_only=True)
nhlTotals = nhl.mean(numeric_only=True)
nhlTotals

Unnamed: 0     16.500000
GP             20.000000
TOI           941.187497
SH%             8.714375
SV%            91.280313
PDO             0.999937
dtype: float64

In [231]:
edmIfVan = edmClean[["Game", "Result", "SF", "SA", "GF", "GA", "xGF", "xGA"]].assign(
    vanSHGF=round((vanTotals["SH%"] / 100) * edmClean["SF"], 1),
    vanSVGA=round((edmClean["SA"] - (vanTotals["SV%"] / 100) * edmClean["SA"]), 1),
    nhlSHGF=round((nhlTotals["SH%"] / 100 * edmClean["SF"]), 1),
    nhlSVGA=round((edmClean["SA"] - (nhlTotals["SV%"] / 100) * edmClean["SA"]), 1))

vanIfEdm = vanClean[["Game", "Result", "SF", "SA", "GF", "GA", "xGF", "xGA"]].assign(
    edmSHGF=round((edmTotals["SH%"] / 100) * vanClean["SF"], 1),
    edmSVGA=round((vanClean["SA"] - (edmTotals["SV%"] / 100) * vanClean["SA"]), 1),
    nhlSHGF=round((nhlTotals["SH%"] / 100 * vanClean["SF"]), 1),
    nhlSVGA=round((vanClean["SA"] - (nhlTotals["SV%"] / 100) * vanClean["SA"]), 1))

vanIfEdm.head()


,Game,Result,SF,SA,GF,GA,xGF,xGA,edmSHGF,edmSVGA,nhlSHGF,nhlSVGA
0,"2023-10-11 - Oilers 1, Canucks 8",W,32,27,8,1,3.83,2.97,3.0,3.7,2.8,2.4
1,"2023-10-14 - Canucks 4, Oilers 3",W,16,40,4,3,2.07,5.16,1.5,5.4,1.4,3.5
2,"2023-10-17 - Canucks 0, Flyers 2",L,26,42,0,2,2.14,4.78,2.5,5.7,2.3,3.7
3,"2023-10-19 - Canucks 3, Lightning 4",L,29,36,3,4,2.22,4.33,2.7,4.9,2.5,3.1
4,"2023-10-21 - Canucks 5, Panthers 3",W,31,37,5,3,3.94,2.88,2.9,5.0,2.7,3.2


In [232]:
edmResults = edmIfVan[["Game", "Result"]].assign(    
    xGSH=np.where(edmIfVan["xGF"] > edmIfVan["GA"], "W", "L"),
    xGSV=np.where(edmIfVan["GF"] > edmIfVan["xGA"], "W", "L"),
    xGPDO=np.where(edmIfVan["xGF"] > edmIfVan["xGA"], "W", "L"),
    vanSH=np.where(edmIfVan["vanSHGF"] > edmIfVan["GA"], "W", "L"),
    vanSV=np.where(edmIfVan["GF"] > edmIfVan["vanSVGA"], "W", "L"),
    vanPDO=np.where(edmIfVan["vanSHGF"] > edmIfVan["vanSVGA"], "W", "L"),
    nhlSH=np.where(edmIfVan["nhlSHGF"] > edmIfVan["GA"], "W", "L"),
    nhlSV=np.where(edmIfVan["GF"] > edmIfVan["nhlSVGA"], "W", "L"),
    nhlPDO=np.where(edmIfVan["nhlSHGF"] > edmIfVan["nhlSVGA"], "W", "L"))

edmResults.head()

,Game,Result,xGSH,xGSV,xGPDO,vanSH,vanSV,vanPDO,nhlSH,nhlSV,nhlPDO
0,"2023-10-11 - Oilers 1, Canucks 8",L,L,L,L,L,L,W,L,L,L
1,"2023-10-14 - Canucks 4, Oilers 3",L,W,W,W,W,W,W,L,W,W
2,"2023-10-17 - Oilers 6, Predators 1",W,W,W,L,W,W,W,W,W,L
3,"2023-10-19 - Oilers 1, Flyers 4",L,L,L,L,L,L,W,L,L,L
4,"2023-10-21 - Jets 3, Oilers 2",L,W,L,W,W,L,W,W,L,W


In [233]:
vanResults = vanIfEdm[["Game", "Result"]].assign(    
    xGSH=np.where(vanIfEdm["xGF"] > vanIfEdm["GA"], "W", "L"),
    xGSV=np.where(vanIfEdm["GF"] > vanIfEdm["xGA"], "W", "L"),
    xGPDO=np.where(vanIfEdm["xGF"] > vanIfEdm["xGA"], "W", "L"),
    edmSH=np.where(vanIfEdm["edmSHGF"] > vanIfEdm["GA"], "W", "L"),
    edmSV=np.where(vanIfEdm["GF"] > vanIfEdm["edmSVGA"], "W", "L"),
    edmPDO=np.where(vanIfEdm["edmSHGF"] > vanIfEdm["edmSVGA"], "W", "L"),
    nhlSH=np.where(vanIfEdm["nhlSHGF"] > vanIfEdm["GA"], "W", "L"),
    nhlSV=np.where(vanIfEdm["GF"] > vanIfEdm["nhlSVGA"], "W", "L"),
    nhlPDO=np.where(vanIfEdm["nhlSHGF"] > vanIfEdm["nhlSVGA"], "W", "L"))
vanResults

,Game,Result,xGSH,xGSV,xGPDO,edmSH,edmSV,edmPDO,nhlSH,nhlSV,nhlPDO
0,"2023-10-11 - Oilers 1, Canucks 8",W,W,W,W,W,W,L,W,W,W
1,"2023-10-14 - Canucks 4, Oilers 3",W,L,L,L,L,L,L,L,W,L
2,"2023-10-17 - Canucks 0, Flyers 2",L,W,L,L,W,L,L,W,L,L
3,"2023-10-19 - Canucks 3, Lightning 4",L,L,L,L,L,L,L,L,L,L
4,"2023-10-21 - Canucks 5, Panthers 3",W,W,W,W,L,L,L,L,W,L
5,"2023-10-24 - Canucks 3, Predators 2",W,W,W,W,W,W,L,W,W,W
6,"2023-10-27 - Blues 0, Canucks 5",W,W,W,W,W,W,W,W,W,W
7,"2023-10-28 - Rangers 4, Canucks 3",L,L,L,W,L,L,L,L,W,W
8,"2023-10-31 - Predators 2, Canucks 5",W,W,W,L,W,W,L,W,W,L
9,"2023-11-02 - Canucks 10, Sharks 1",W,W,W,W,W,W,L,W,W,W


In [234]:
edmRecord = edmResults.apply(lambda x: x.value_counts()).fillna(0)
edmRecord = edmRecord.loc[["W", "L"]].drop("Game", axis=1)
vanRecord = vanResults.apply(lambda x: x.value_counts()).fillna(0)
vanRecord = vanRecord.loc[["W", "L"]].drop("Game", axis=1)


In [235]:
edmResults.loc["Final Results"] = edmRecord.apply(lambda x: f'{round(x.W)}-{round(x.L)}')
vanResults.loc["Final Results"] = vanRecord.apply(lambda x: f'{round(x.W)}-{round(x.L)}')
edmResults

,Game,Result,xGSH,xGSV,xGPDO,vanSH,vanSV,vanPDO,nhlSH,nhlSV,nhlPDO
0,"2023-10-11 - Oilers 1, Canucks 8",L,L,L,L,L,L,W,L,L,L
1,"2023-10-14 - Canucks 4, Oilers 3",L,W,W,W,W,W,W,L,W,W
2,"2023-10-17 - Oilers 6, Predators 1",W,W,W,L,W,W,W,W,W,L
3,"2023-10-19 - Oilers 1, Flyers 4",L,L,L,L,L,L,W,L,L,L
4,"2023-10-21 - Jets 3, Oilers 2",L,W,L,W,W,L,W,W,L,W
5,"2023-10-24 - Oilers 4, Wild 7",L,L,L,L,L,W,W,L,W,L
6,"2023-10-26 - Rangers 3, Oilers 0",L,W,L,W,W,L,W,L,L,L
7,"2023-10-29 - Flames 2, Oilers 5",W,W,W,W,W,W,W,W,W,W
8,"2023-11-02 - Stars 4, Oilers 3",L,W,W,W,W,W,W,W,W,W
9,"2023-11-04 - Predators 5, Oilers 2",L,L,L,L,L,L,W,L,L,W


In [236]:
vanResults

,Game,Result,xGSH,xGSV,xGPDO,edmSH,edmSV,edmPDO,nhlSH,nhlSV,nhlPDO
0,"2023-10-11 - Oilers 1, Canucks 8",W,W,W,W,W,W,L,W,W,W
1,"2023-10-14 - Canucks 4, Oilers 3",W,L,L,L,L,L,L,L,W,L
2,"2023-10-17 - Canucks 0, Flyers 2",L,W,L,L,W,L,L,W,L,L
3,"2023-10-19 - Canucks 3, Lightning 4",L,L,L,L,L,L,L,L,L,L
4,"2023-10-21 - Canucks 5, Panthers 3",W,W,W,W,L,L,L,L,W,L
5,"2023-10-24 - Canucks 3, Predators 2",W,W,W,W,W,W,L,W,W,W
6,"2023-10-27 - Blues 0, Canucks 5",W,W,W,W,W,W,W,W,W,W
7,"2023-10-28 - Rangers 4, Canucks 3",L,L,L,W,L,L,L,L,W,W
8,"2023-10-31 - Predators 2, Canucks 5",W,W,W,L,W,W,L,W,W,L
9,"2023-11-02 - Canucks 10, Sharks 1",W,W,W,W,W,W,L,W,W,W
